## 深圳整租成交数据集

### 背景描述
链家网深圳市整租房源成交数据
- 爬取时间: 20200629 
- 样本数据:1666条
- 样本数据成交最早时间: 20200612
![链家App](QQ20200629.png)

### 数据说明

字段 | 说明
-|-
title | 房源标题
location | 房源位置
price | 房源成交价(元/月)

### 代码实现

#### 导入需要的库

In [2]:
import numpy as np
import pandas as pd
import re
from pyecharts.charts import Map,Geo
from pyecharts import options as opts

df = pd.read_csv('result.csv',encoding='utf-8')
df.head()

,_id,location,title,price
0,"ObjectId(""5ef8a7939b57885c8c5fde4a"")",南山区·前海,整租·星海名城一期四组团 2室1厅,6800
1,"ObjectId(""5ef8a7949b57885c8c5fde4c"")",宝安区·翻身 距1号线(罗宝线)-宝安中心站1km,整租·卡罗社区 1室1厅,5300
2,"ObjectId(""5ef8a7949b57885c8c5fde4f"")",南山区·深圳湾,整租·鸿威海怡湾 3室2厅,14000
3,"ObjectId(""5ef8a7949b57885c8c5fde51"")",龙华区·龙华中心 距4号线(龙华线)-龙华站913m,整租·鸿荣源·壹城中心第九区 4室2厅,7100
4,"ObjectId(""5ef8a7949b57885c8c5fde53"")",罗湖区·百仕达 距3号线(龙岗线)-翠竹站304m,整租·翠竹苑 2室1厅,4200


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       1666 non-null   object
 1   location  1666 non-null   object
 2   title     1666 non-null   object
 3   price     1666 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 52.2+ KB


所有字段无缺失值

### 数据处理

#### 新建字段：trans_list
各行政区整租交易量

In [4]:
df['district'] = df['location'].map(lambda x:(re.compile('(.+?)·').search(x).group(1)).strip())
df_dist = df['district'].value_counts().rename_axis('districts').reset_index(name='counts')
locate = df_dist['districts']
transaction = df_dist['counts']
trans_list= [[locate[i],int(transaction[i])] for i in range(len(locate))]
df_dist

,districts,counts
0,南山区,421
1,龙岗区,357
2,福田区,297
3,宝安区,218
4,龙华区,156
5,罗湖区,150
6,盐田区,32
7,光明区,18
8,坪山区,17


### 可视化 (用 map)
问题是：行政区域重新划分导致大鹏不在地图里面
数据有大鹏区也不报错

In [5]:
LIANJIA_LINK = "https://www.lianjia.com/"
map_1 = Map()
map_1.set_global_opts(
    title_opts=opts.TitleOpts(
        title="深圳各行政区整租市场热力图",
        subtitle="数据来源链家\n统计时间：2020-06-29",
        subtitle_link=LIANJIA_LINK),
    visualmap_opts=opts.VisualMapOpts(
        min_=transaction.min(), 
        max_=transaction.max(),
        range_text=["High", "Low"],
        is_calculable=True,
        range_color=["lightskyblue", "yellow", "orangered"],)  
    )
map_1.add("深圳市场整租成交量", trans_list, "深圳",is_map_symbol_show=False)
map_1.render_notebook()

/Users/xusanshan/Anaconda/anaconda3/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


可以看出

2020年6月12号至6月29号，链家网上深圳整租成交数据显示

- 整租成交量中，排名前三的是南山区(421次),龙岗区(357次)，福田区（297次）
- 整租成交量中，坪山区是最低的，达17次

### 用 geo (跑不通)

不能用自定义的地图

geo.add_coordinate 只是添加坐标，地图行政区划分仍然没变

In [5]:

LIANJIA_LINK = "https://www.lianjia.com/"

geo = Geo()

geo.add_schema(maptype="深圳",is_roam= False)
geo.add("深圳市场整租成交量", trans_list)
geo.set_global_opts(
    title_opts=opts.TitleOpts(
        title="深圳各行政区整租市场热力图",
        subtitle="数据来源：链家\n统计时间：2020-06-29",
        subtitle_link=LIANJIA_LINK),
    visualmap_opts=opts.VisualMapOpts(
        min_=transaction.min(), 
        max_=transaction.max(),
        range_text=["High", "Low"],
        is_calculable=True,
        range_color=["lightskyblue", "yellow", "orangered"],)  
    )
#geo.add_coordinate( "大鹏新区",114.48,22.59)
geo.add_coordinate_json("my_coords.json")
geo.render_notebook()

/Users/xusanshan/Anaconda/anaconda3/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### 地图 json

制作地图json
和 js 文件

[pyecharts 自定义地图之添加js文件](https://blog.csdn.net/wbgljy520/article/details/87934369) 无效
[老版 pyecharts 教程](https://pyecharts.readthedocs.io/zh/latest/zh-cn/customize_map/)
更换 pyecharts.datasets.map_filenames.json 文件对对应 深圳的js文件，也无效
可能是 pyecharts 资源引用从服务器来，改本地文件没有

In [8]:


LIANJIA_LINK = "https://www.lianjia.com/"
map_1 = Map()
map_1.set_global_opts(
    title_opts=opts.TitleOpts(
        title="深圳各行政区整租市场热力图",
        subtitle="数据来源链家\n统计时间：2020-06-29",
        subtitle_link=LIANJIA_LINK),
    visualmap_opts=opts.VisualMapOpts(
        min_=transaction.min(), 
        max_=transaction.max(),
        range_text=["High", "Low"],
        is_calculable=True,
        range_color=["lightskyblue", "yellow", "orangered"],)  
    )
map_1.add("深圳市场整租成交量", trans_list, "深圳",is_map_symbol_show=False)
map_1.render_notebook()

/Users/xusanshan/Anaconda/anaconda3/lib/python3.6/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [9]:
import pyecharts

pyecharts.__version__

'1.8.1'